In [109]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split as tts

In [123]:
df = pd.read_csv('../Data/train.csv')

In [124]:
df.drop(columns=['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'accommodates', 'bathrooms'
       , 'beds', 'amenities', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'last_review', 'license',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month','host_since','bathrooms_text'], inplace=True)


In [125]:
def bath_clean(s):
    if re.findall('(?i)(shared)|(half)',str(s)):
        return 0
    
    else:
        if re.findall('\d',str(s)): return re.findall('\d',s)[0]
        else: return 0

In [126]:
def to_num(df):
    	
    df.host_is_superhost = df.host_is_superhost.apply(lambda x: 1 if x=='t' else 0)
    df.instant_bookable = df.instant_bookable.apply(lambda x: 1 if x=='t' else 0)
    df.host_has_profile_pic=df.host_has_profile_pic.apply(lambda x: 1 if x=='t' else 0)
    df.has_availability = df.has_availability.apply(lambda x: 0 if x=='t' else 1)

    #df.host_since = df.host_since.apply(lambda x: 2022-int(re.findall('\d{4}',x)[0]))
    # df.bathrooms_text = df.bathrooms_text.apply(bath_clean)


    # df['lang_des']=detect_lan(df, 'description')
    # df['lang_host']=detect_lan(df, 'host_about')
    # df.drop(columns=['host_about','description'])

    df[[i.lower().replace(' ','_') for i in df.room_type.unique()]] = pd.get_dummies(df.room_type)
    df.drop(columns=['room_type'], inplace=True)
    


    return df
df = to_num(df)

In [127]:
df

,host_is_superhost,host_has_profile_pic,bedrooms,price,has_availability,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,private_room,entire_home/apt,hotel_room,shared_room
0,0,1,1.0,87.0,0,4.62,4.56,4.29,4.57,4.75,4.69,4.49,0,0,0,1,0
1,0,1,2.0,250.0,0,5.00,5.00,5.00,4.89,5.00,5.00,4.89,0,1,0,0,0
2,1,1,1.0,151.0,0,4.79,4.83,4.91,4.62,4.87,4.87,4.40,1,0,1,0,0
3,0,1,1.0,139.0,0,5.00,5.00,4.80,5.00,5.00,4.90,4.70,0,1,0,0,0
4,1,1,1.0,95.0,0,4.67,4.76,4.81,4.95,4.89,4.49,4.57,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,0,1,1.0,95.0,0,4.93,5.00,4.71,5.00,4.86,4.86,4.79,0,1,0,0,0
4163,0,1,1.0,65.0,0,5.00,5.00,5.00,5.00,5.00,4.50,5.00,1,1,0,0,0
4164,1,1,2.0,114.0,0,5.00,4.80,4.80,5.00,5.00,5.00,4.60,0,1,0,0,0
4165,1,1,1.0,416.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,1,0


In [128]:
df.dropna(inplace=True)

In [129]:
X_train, X_test, y_train, y_test = tts(df.drop('price', axis=1), df.price) 

In [130]:
def regre(modelo):
    
    modelo.fit(X_train, y_train)
    
    train_score=modelo.score(X_train, y_train)  # R2
    test_score=modelo.score(X_test, y_test)
    
    print(modelo)
    print('Train R2:', train_score)
    print('Test R2:', test_score)
    
    return modelo

In [131]:
from sklearn.ensemble import RandomForestRegressor as RFR
rfr=RFR(n_estimators=250)

In [132]:
rfr.fit(df.drop('price', axis=1), df.price)

RandomForestRegressor(n_estimators=250)

In [134]:
df_test = pd.read_csv('../Data/test.csv')

In [135]:
df_test.drop(columns=['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'accommodates', 'bathrooms'
       , 'beds', 'amenities', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'first_review', 'last_review', 'license',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month','host_since','bathrooms_text'], inplace=True)

In [136]:
df_test.fillna(0, inplace=True)
df_test = to_num(df_test)

In [137]:
y_pred = rfr.predict(df_test)

/home/pomilusky/.local/lib/python3.8/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.



In [138]:
y_pred

array([119.03211005, 105.644     , 154.504     , ..., 206.9       ,
       158.11775123,  87.48      ])

In [139]:
df_sample = pd.read_csv('../Data/sample.csv')

In [142]:
df_sample['price']=y_pred
df_sample.min()

price    54.14
id        0.00
dtype: float64

In [76]:
df_sample.to_csv('../Data/resultats.csv')